<a href="https://colab.research.google.com/github/ashika12344/Movie-Recommendations-/blob/main/ai_movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages
!pip install gradio pandas scikit-learn

# Import libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# Load dataset from GitHub (raw CSV file)
csv_url = "https://raw.githubusercontent.com/ashika12344/AI-Movi-Recommendaton/refs/heads/main/movies_data.csv"
df = pd.read_csv(csv_url, on_bad_lines='skip')

# Prepare the dataset
df['title_lower'] = df['title'].str.lower()
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['genres'].fillna(''))
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df.index, index=df['title_lower']).drop_duplicates()

# Define recommendation function
def recommend_movie(title):
    title_lower = title.lower().strip()
    if title_lower not in indices:
        return "❌ Movie not found. Please try another title."

    idx = indices[title_lower]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]

    result = ""
    for i in movie_indices:
        result += f"🎬 **{df['title'].iloc[i]}**\n"
        result += f"⭐ Rating: {df['rating'].iloc[i]}/10\n"
        result += f"🎭 Genre: {df['genres'].iloc[i]}\n"
        result += f"📝 Tagline: _{df['tagline'].iloc[i]}_\n\n"

    return result.strip()

# Launch Gradio app with 'Deadpool' pre-filled
gr.Interface(
    fn=recommend_movie,
    inputs=gr.Textbox(lines=1, label="🎥 Enter a Movie Title", value="Deadpool"), # Pre-fill here
    outputs=gr.Markdown(label="🎯 Recommended Movies with Details"),
    title="🎬 AI Movie Recommender",
    description="Enter a movie title to get personalized recommendations including ratings and taglines."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c2002a564687770822.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
